In [12]:
import requests
import json
from config import Api_key
# from fpdf import FPDF
import smtplib
# from email.mime.text import MIMEText
# from email.mime.multipart import MIMEMultipart
# from email.mime.application import MIMEApplication

In [13]:
# Mistral API Config
API_KEY = Api_key
API_URL = "https://api.mistral.ai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}


In [14]:

# Exemple de données CV au format JSON
cv_data = {
    "full_name": "Jean Dupont",
    "email": "jean.dupont@email.com",
    "experiences": [{"title": "Développeur Python", "company": "TechCorp", "start_date": "2021-05", "end_date": "2023-04"}],
    "education": [{"school": "Université de Lyon", "degree": "Master Informatique", "year": "2020"}],
    "skills": ["Python", "Flask", "SQL"],
    "certifications": ["AWS Certified Developer"],
    "languages": ["Français (natif)", "Anglais (courant)"]
}


In [16]:

def appeler_mistral(cv, domaine, ville):
    prompt = f"""
    Tu es un conseiller d'orientation. Voici le profil :
    {json.dumps(cv, indent=4)}

    Il souhaite se former dans le domaine : {domaine}, à {ville}.

    Fournis 5 écoles ou formations en france qui correspondent à ce profil et a les meilleurs avis:

    * Nom
    * Ville
    * Type (mastère, reconversion…)
    * Justification
    * Site officiel (lien si possible)
    """

    payload = {
        "model": "mistral-medium",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=HEADERS, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

In [17]:
print("🎯 Agent IA : Écoles recommandées")
domaine = input("👉 Quel domaine veux-tu étudier ? : ")
ville = input("📍 Où veux-tu te former ? : ")

print("\n🔍 Recherche en cours...\n")
resultat = appeler_mistral(cv_data, domaine, ville)

print("📋 Résultats :\n")
print(resultat)

🎯 Agent IA : Écoles recommandées

🔍 Recherche en cours...

📋 Résultats :

Voici 5 formations en mathématiques à Angers ou en France qui pourraient correspondre au profil de Jean Dupont, en tenant compte de son expérience en informatique et de sa volonté de se former en mathématiques :

### 1. **Master Mathématiques Appliquées, Statistique**
   - **Ville** : Angers
   - **Type** : Master (Bac+5)
   - **Justification** : Ce master propose une formation solide en mathématiques appliquées, avec des cours en statistique et modélisation, complémentaires à ses compétences en Python et SQL. Idéal pour une transition vers la data science ou l'analyse quantitative.
   - **Site officiel** : [Université d'Angers - Master Maths Appliquées](https://www.univ-angers.fr/fr/formation/master-maths-appliquees.html)

### 2. **Licence Professionnelle Métiers de l'Informatique : Applications Web (avec parcours Maths-Info)**
   - **Ville** : Angers
   - **Type** : Licence Pro (Bac+3, reconversion possible)
  

In [ ]:
def creer_pdf(resultat, filename="ecoles_recommandees.pdf"):
    pdf = FPDF()
    pdf.add_page()

    # Utilisation de la police Arial par défaut
    pdf.set_font("Arial", size=12)

    # Nettoyage du texte avant l'ajout (remplacement des caractères non encodables)
    resultat = resultat.replace("\u2013", "-")  # Remplacement des tirets longs (en dash)
    
    for ligne in resultat.split("\n"):
        pdf.multi_cell(0, 10, ligne)

    pdf.output(filename)
    return filename



In [ ]:
def envoyer_email(destinataire, sujet, corps, fichier_pdf):
    expediteur = "ton.email@exemple.com"
    mot_de_passe = "TON_MOT_DE_PASSE_EMAIL"

    msg = MIMEMultipart()
    msg["From"] = expediteur
    msg["To"] = destinataire
    msg["Subject"] = sujet

    msg.attach(MIMEText(corps, "plain"))

    with open(fichier_pdf, "rb") as f:
        pdf_part = MIMEApplication(f.read(), _subtype="pdf")
        pdf_part.add_header('Content-Disposition', 'attachment', filename=fichier_pdf)
        msg.attach(pdf_part)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(expediteur, mot_de_passe)
        server.send_message(msg)



In [6]:
def main():
    print("🎯 Agent IA : Écoles recommandées")
    domaine = input("👉 Quel domaine veux-tu étudier ? : ")
    ville = input("📍 Où veux-tu te former ? : ")

    print("\n🔍 Recherche en cours...\n")
    resultat = appeler_mistral(cv_data, domaine, ville)

    print("📋 Résultats :\n")
    print(resultat)

    # # Création du PDF avec les résultats
    # nom_pdf = creer_pdf(resultat)
    # print(f"\n📄 Résultat enregistré dans le fichier : {nom_pdf}")

    # # Envoi par email (optionnel)
    # envoyer = input("✉ Veux-tu recevoir le résultat par email ? (o/n) : ").lower()
    # if envoyer == "o":
    #     email_dest = input("📧 Ton adresse email : ")
    #     envoyer_email(email_dest, "Tes écoles recommandées", "Voici le PDF avec les écoles.", nom_pdf)
    #     print("✅ Email envoyé !")
